In [1]:
print("Hello World!")

Hello World!


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

# Try reading the CSV file with a different encoding
try:
    store = pd.read_csv("Sample-Superstore.csv", encoding='latin-1')
except UnicodeDecodeError:
    print("Error: Unable to read the file with 'latin-1' encoding.")
    try:
        store = pd.read_csv("Sample-Superstore.csv", encoding='utf-16')
    except UnicodeDecodeError:
        print("Error: Unable to read the file with 'utf-16' encoding.")

In [3]:

## renaming of the columns to get rid of the space in between column names 
store.rename(columns={"Order ID": "OrderID", "Order Date": "OrderDate", "Ship Date": "ShipDate", "Ship Mode": "ShipMode", "Customer ID": "CustomerID", "Customer Name": "CustomerName", "Postal Code": "PostalCode", "Product ID": "ProductID","Sub-Category":"SubCategory", "Product Name": "ProductName"}, inplace=True)


In [4]:
store_copied= store.copy()

## Creating  new columns

In [5]:


## ProductPrice is the vPrrice per unit of Product  is sold
store_copied['ProductPrice'] = ((store_copied['Sales'] / store_copied['Quantity'])/ (1-store_copied['Discount'])).round(2)

## SupplierPrice is the Prrice per unit of Product  is bought
store_copied['SupplierPrice'] = ((store_copied['Sales'] - store_copied['Profit']) / store_copied['Quantity']).round(2)





In [6]:
#Sales, Profit and Supplier Price are funky as heck. They need to be rounded to the nearest cent. 
#All prices will also need to be in CENTS to avoid floating point errors in SQLite.

cols = ["Sales", "Profit", "SupplierPrice", "ProductPrice"]

store_copied.loc[:, cols] = (store_copied[cols]*100).astype(int)

In [7]:
#We need to fix OrderDate and ship date to be YYYY/MM/DD (can be ordered easily even when seen as a string). 
store_copied[['OrderDate','ShipDate']] = store_copied[['OrderDate','ShipDate']].apply(pd.to_datetime)

In [17]:
store_copied.head(3)

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26195.0,2,0.0,4191.0,13097.0,11002.0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0.0,21958.0,24398.0,17079.0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.0,687.0,731.0,387.0


# Conversion of DataFrame into Component Table

In [ ]:
# First pass

customers = store_copied[["CustomerID", "CustomerName", "Segment"]]
address = store_copied[["CustomerID","Country", "Region", "State", "City", "PostalCode"]] # Address ID will be required to connect this to orders. 
products = store_copied[["ProductID", "ProductName","Category", "SubCategory", "ProductPrice", "SupplierPrice"]] 
orders = store_copied[["OrderID", "CustomerID", "OrderDate", "ShipDate", "ShipMode"]] #Needs Address ID!
orderDetails = store_copied[["OrderID", "ProductID", "Discount", "Quantity"]] # Order_detailsID will be required to connect this to orders and products. 

## Customers

In [8]:
cus=store_copied[["CustomerID", "CustomerName","Segment"]]

In [9]:
## dropping duplicates

cus.drop_duplicates(inplace=True)
customer_f=cus.reset_index(drop=True)

C:\Users\godwi\AppData\Local\Temp\ipykernel_21808\1124437389.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus.drop_duplicates(inplace=True)


In [10]:
customers=customer_f[["CustomerID","CustomerName","Segment"]]

In [11]:
customers

,CustomerID,CustomerName,Segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer
...,...,...,...
788,CJ-11875,Carl Jackson,Corporate
789,RS-19870,Roy Skaria,Home Office
790,SC-20845,Sung Chung,Consumer
791,RE-19405,Ricardo Emerson,Consumer


## Address

In [12]:
address_dff=store_copied[["CustomerID","Country","Region","State","City","PostalCode"]]

In [13]:
#The goal here is to see if customers have multiple addresses.
address_dff.groupby(['CustomerID', 'PostalCode']).ngroups

4910

In [14]:
address_df=address_dff.drop_duplicates()

In [15]:
duplicate = address_df[address_df.duplicated('CustomerID')]

In [16]:
duplicate.sort_values(by='CustomerID', ascending=False)

,CustomerID,Country,Region,State,City,PostalCode
3814,ZD-21925,United States,South,Florida,Jacksonville,32216
3040,ZD-21925,United States,South,Tennessee,Chattanooga,37421
5897,ZD-21925,United States,Central,Indiana,Richmond,47374
8341,ZD-21925,United States,West,California,Los Angeles,90036
8923,ZC-21910,United States,South,North Carolina,Hickory,28601
...,...,...,...,...,...,...
807,AA-10375,United States,Central,Nebraska,Omaha,68104
1299,AA-10315,United States,West,California,San Francisco,94109
2229,AA-10315,United States,West,California,San Francisco,94122
7468,AA-10315,United States,East,New York,New York City,10011


In [17]:
#We need to make a new column to use as a primary key for Addresses.
address_df['AddressID'] = range(1, len(address_df) + 1)

C:\Users\godwi\AppData\Local\Temp\ipykernel_21808\2491359544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_df['AddressID'] = range(1, len(address_df) + 1)


In [18]:
address_df['AddressID'] = "A" + address_df['AddressID'].astype(str).str.zfill(6)

C:\Users\godwi\AppData\Local\Temp\ipykernel_21808\1134945338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_df['AddressID'] = "A" + address_df['AddressID'].astype(str).str.zfill(6)


In [19]:
address_f= address_df

In [20]:
address_f.reset_index(drop=True)

,CustomerID,Country,Region,State,City,PostalCode,AddressID
0,CG-12520,United States,South,Kentucky,Henderson,42420,A000001
1,DV-13045,United States,West,California,Los Angeles,90036,A000002
2,SO-20335,United States,South,Florida,Fort Lauderdale,33311,A000003
3,BH-11710,United States,West,California,Los Angeles,90032,A000004
4,AA-10480,United States,South,North Carolina,Concord,28027,A000005
...,...,...,...,...,...,...,...
4905,ML-17410,United States,West,California,Los Angeles,90008,A004906
4906,RA-19885,United States,South,Georgia,Athens,30605,A004907
4907,TB-21400,United States,South,Florida,Miami,33180,A004908
4908,DB-13060,United States,West,California,Costa Mesa,92627,A004909


## Orders

In [21]:
address_f.columns

Index(['CustomerID', 'Country', 'Region', 'State', 'City', 'PostalCode',
       'AddressID'],
      dtype='object')

In [22]:
dfs2 = address_f.merge(store_copied, left_on=["CustomerID","Country","Region","State","City","PostalCode"], right_on=["CustomerID","Country","Region","State","City","PostalCode"], how='inner').sort_values(by="Row ID", ignore_index=True)

In [23]:
store_order=dfs2.copy()

In [24]:
ord=store_order[["OrderID","OrderDate","ShipDate","ShipMode","AddressID","CustomerID"]] 

In [25]:
ord[["OrderID","OrderDate","ShipDate","ShipMode","AddressID","CustomerID"]].nunique()

OrderID       5009
OrderDate     1237
ShipDate      1334
ShipMode         4
AddressID     4910
CustomerID     793
dtype: int64

In [26]:
ord[["OrderID","OrderDate","ShipDate","ShipMode","CustomerID"]].duplicated().sum()

4985

In [27]:
store_order_no_duplicates = store_order.drop_duplicates(subset=['OrderID'])
store_order_no_duplicates.reset_index(drop=True)

,CustomerID,Country,Region,State,City,PostalCode,AddressID,Row ID,OrderID,OrderDate,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,CG-12520,United States,South,Kentucky,Henderson,42420,A000001,1,CA-2016-152156,2016-11-08,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26195.0,2,0.00,4191.0,13097.0,11002.0
1,DV-13045,United States,West,California,Los Angeles,90036,A000002,3,CA-2016-138688,2016-06-12,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.00,687.0,731.0,387.0
2,SO-20335,United States,South,Florida,Fort Lauderdale,33311,A000003,4,US-2015-108966,2015-10-11,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,95757.0,5,0.45,-38303.0,34821.0,26812.0
3,BH-11710,United States,West,California,Los Angeles,90032,A000004,6,CA-2014-115812,2014-06-09,...,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,4886.0,7,0.00,1416.0,698.0,496.0
4,AA-10480,United States,South,North Carolina,Concord,28027,A000005,13,CA-2017-114412,2017-04-15,...,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,1555.0,3,0.20,544.0,648.0,337.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,ML-17410,United States,West,California,Los Angeles,90008,A004906,9987,CA-2016-125794,2016-09-29,...,TEC-AC-10003399,Technology,Accessories,Memorex Mini Travel Drive 64 GB USB 2.0 Flash ...,3624.0,1,0.00,1522.0,3624.0,2102.0
5005,RA-19885,United States,South,Georgia,Athens,30605,A004907,9988,CA-2017-163629,2017-11-17,...,TEC-AC-10001539,Technology,Accessories,Logitech G430 Surround Sound Gaming Headset wi...,7998.0,1,0.00,2879.0,7998.0,5119.0
5006,TB-21400,United States,South,Florida,Miami,33180,A004908,9990,CA-2014-110422,2014-01-21,...,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,2524.0,3,0.20,410.0,1052.0,705.0
5007,DB-13060,United States,West,California,Costa Mesa,92627,A004909,9991,CA-2017-121258,2017-02-26,...,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,9196.0,2,0.00,1563.0,4598.0,3815.0


In [28]:
orders= store_order_no_duplicates[["OrderID","OrderDate","ShipDate","ShipMode","AddressID","CustomerID"]] 

In [29]:
orders = store_order_no_duplicates.sort_values('OrderDate', ascending=True, ignore_index=True)

In [30]:
orders = store_order_no_duplicates.sort_values('OrderDate', ascending=True, ignore_index=True)

In [31]:
orders

,CustomerID,Country,Region,State,City,PostalCode,AddressID,Row ID,OrderID,OrderDate,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,DP-13000,United States,Central,Texas,Houston,77095,A003887,7981,CA-2014-103800,2014-01-03,...,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",1644.0,2,0.2,555.0,1028.0,545.0
1,PO-19195,United States,Central,Illinois,Naperville,60540,A000343,740,CA-2014-112326,2014-01-04,...,OFF-LA-10003223,Office Supplies,Labels,Avery 508,1178.0,3,0.2,427.0,491.0,250.0
2,MB-18085,United States,East,Pennsylvania,Philadelphia,19143,A000854,1760,CA-2014-141817,2014-01-05,...,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,1953.0,3,0.2,488.0,814.0,488.0
3,JO-15145,United States,South,Georgia,Athens,30605,A003509,7181,CA-2014-106054,2014-01-06,...,OFF-AR-10002399,Office Supplies,Art,"Dixon Prang Watercolor Pencils, 10-Color Set w...",1278.0,3,0.0,523.0,426.0,250.0
4,LS-17230,United States,West,California,Los Angeles,90049,A002614,5328,CA-2014-130813,2014-01-06,...,OFF-PA-10002005,Office Supplies,Paper,Xerox 225,1944.0,3,0.0,933.0,648.0,337.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,BS-11755,United States,West,Washington,Edmonds,98026,A002686,5457,CA-2017-130631,2017-12-29,...,OFF-FA-10000089,Office Supplies,Fasteners,Acco Glide Clips,1960.0,5,0.0,960.0,392.0,200.0
5005,PO-18865,United States,East,New York,New York City,10009,A000432,907,CA-2017-143259,2017-12-30,...,FUR-BO-10003441,Furniture,Bookcases,"Bush Westfield Collection Bookcases, Fully Ass...",32313.0,4,0.2,1211.0,10098.0,7775.0
5006,EB-13975,United States,West,California,Fairfield,94533,A000610,1297,CA-2017-115427,2017-12-30,...,OFF-BI-10002103,Office Supplies,Binders,"Cardinal Slant-D Ring Binder, Heavy Gauge Vinyl",1390.0,2,0.2,451.0,869.0,469.0
5007,CC-12430,United States,Central,Indiana,Columbus,47201,A000300,646,CA-2017-126221,2017-12-30,...,OFF-AP-10002457,Office Supplies,Appliances,Eureka The Boss Plus 12-Amp Hard Box Upright V...,20930.0,2,0.0,5651.0,10465.0,7639.0


## Products

In [32]:
store_product=store_copied.copy()

In [33]:
#Drop duplicate ProductIDs
store_product = store_product.drop_duplicates(subset=['ProductID', "ProductName"])

In [34]:
store_product_df=store_product.reset_index(drop=True)

In [35]:
store_product_df

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,26195.0,2,0.00,4191.0,13097.0,11002.0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",73194.0,3,0.00,21958.0,24398.0,17079.0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1462.0,2,0.00,687.0,731.0,387.0
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,95757.0,5,0.45,-38303.0,34821.0,26812.0
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,2236.0,2,0.20,251.0,1398.0,993.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,9522,CA-2014-169446,2014-12-19,2014-12-25,Standard Class,SG-20605,Speros Goranitis,Consumer,United States,Chicago,...,TEC-PH-10002817,Technology,Phones,RCA ViSYS 25425RE1 Corded phone,32397.0,3,0.20,3644.0,13499.0,9584.0
1890,9563,CA-2016-129280,2016-05-03,2016-05-05,First Class,SM-20905,Susan MacKendrick,Consumer,United States,Newark,...,TEC-MA-10003589,Technology,Machines,Cisco 8961 IP Phone Charcoal,22493.0,3,0.70,-16495.0,24993.0,12996.0
1891,9605,CA-2015-138625,2015-11-02,2015-11-05,First Class,EG-13900,Emily Grady,Consumer,United States,Chesapeake,...,OFF-AP-10003099,Office Supplies,Appliances,"Eureka Hand Vacuum, Bagless",19772.0,4,0.00,5536.0,4943.0,3559.0
1892,9674,CA-2016-114867,2016-12-23,2016-12-28,Standard Class,FM-14290,Frank Merwin,Home Office,United States,Philadelphia,...,TEC-PH-10002645,Technology,Phones,LG G2,149997.0,5,0.40,-37499.0,49999.0,37499.0


In [36]:
#Here I want to see if the prices remain consistent throughout the dataset. 
#Futureprooding would definitely require us to be able to change Product or Supplier price going forward.
price_test = store_product_df[["ProductID","ProductPrice","SupplierPrice"]]

In [37]:
price_test.sort_values('ProductID')

,ProductID,ProductPrice,SupplierPrice
1567,FUR-BO-10000112,13097.0,10478.0
1129,FUR-BO-10000330,12098.0,10404.0
1066,FUR-BO-10000362,17098.0,13165.0
1205,FUR-BO-10000468,4858.0,4081.0
1651,FUR-BO-10000711,7098.0,5182.0
...,...,...,...
1535,TEC-PH-10004912,5495.0,3901.0
1096,TEC-PH-10004922,6698.0,5024.0
1189,TEC-PH-10004924,739.0,539.0
1171,TEC-PH-10004959,10049.0,7537.0


In [38]:
store_product_df[store_product_df.groupby('ProductID')['ProductID'].transform('size') > 1].sort_values(by=['ProductID'])

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
1364,2472,US-2016-135923,2016-01-22,2016-01-28,Standard Class,CM-11935,Carlos Meador,Consumer,United States,Fayetteville,...,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish",45113.0,4,0.20,-6767.0,14097.0,12969.0
1259,2116,CA-2015-164882,2015-10-31,2015-10-31,Same Day,SG-20080,Sandra Glassco,Consumer,United States,Redlands,...,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases,42583.0,1,0.15,2003.0,50098.0,40579.0
64,67,US-2015-164175,2015-04-30,2015-05-05,Standard Class,PS-18970,Paul Stevenson,Home Office,United States,Chicago,...,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray",21311.0,5,0.30,-1522.0,6089.0,4567.0
124,129,US-2016-125969,2016-11-06,2016-11-10,Second Class,LS-16975,Lindsay Shagiari,Home Office,United States,Los Angeles,...,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black",8142.0,2,0.20,-916.0,5089.0,4529.0
1010,1460,CA-2014-155271,2014-05-04,2014-05-04,Same Day,AA-10480,Andrew Allen,Consumer,United States,Middletown,...,FUR-FU-10001473,Furniture,Furnishings,DAX Wood Document Frame,2746.0,2,0.00,988.0,1373.0,878.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,1220,US-2017-118087,2017-09-09,2017-09-13,Standard Class,SP-20620,Stefania Perrino,Corporate,United States,Philadelphia,...,TEC-PH-10002200,Technology,Phones,Samsung Galaxy Note 2,193195.0,7,0.40,-38639.0,45999.0,33119.0
1399,2597,CA-2017-149048,2017-05-13,2017-05-17,Standard Class,BM-11650,Brian Moss,Corporate,United States,Columbus,...,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...,58797.0,3,0.00,15875.0,19599.0,14307.0
974,1379,CA-2014-126361,2014-08-04,2014-08-09,Second Class,VD-21670,Valerie Dominguez,Consumer,United States,Pleasant Grove,...,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone,39996.0,5,0.20,3499.0,9999.0,7298.0
727,923,CA-2015-111164,2015-04-11,2015-04-15,Standard Class,SE-20110,Sanjit Engle,Consumer,United States,New York City,...,TEC-PH-10004531,Technology,Phones,OtterBox Commuter Series Case - iPhone 5 & 5s,2199.0,1,0.00,1055.0,2199.0,1143.0


In [39]:
store_product_df[store_product_df.groupby('ProductID')['ProductID'].transform('size') > 2].sort_values(by=['ProductID'])

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice


In [ ]:
#So we have a bunch of products who share IDs and all do seem to belong in the correct category.
#So we must manage them with as little fuss as possible. If stock numbers were known then we would change ones with lower num. items in stock (less work)
#But we don't know this so let's modify every second one. As we can see above, never more than one duplicate.

In [40]:
#Let's find the biggest value in the column and work from there. 
#I imagine all products are added iteratively i.e. a new product is assigned the next available number regardless of category and subcategory.
store_product_df['ProductID'].str[-4:].max()

'4999'

In [41]:
# Sort the DataFrame based on 'ProductID' to make sure the IDs are in order
store_product_df.sort_values('ProductID', inplace=True)

# Calculate the current_count based on the maximum value of the last four digits
current_count = int(store_product_df['ProductID'].str[-4:].max())

# Iterate through the DataFrame and update the duplicated ProductIDs
for index, row in store_product_df.iterrows():
    product_id = row['ProductID']
    
    # Check if the current ProductID is a duplicate
    if store_product_df[store_product_df['ProductID'] == product_id].shape[0] > 1:
        # Increment the count and update the ProductID
        current_count += 1
        new_product_id = f"{product_id[:-4]}{current_count:04d}"
        store_product_df.loc[index, 'ProductID'] = new_product_id

# Reset the index
store_product_df.reset_index(drop=True, inplace=True)


In [42]:
store_product_df

,Row ID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,...,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,ProductPrice,SupplierPrice
0,3513,CA-2017-140326,2017-09-04,2017-09-06,First Class,HW-14935,Helen Wasserman,Corporate,United States,Chicago,...,FUR-BO-10000112,Furniture,Bookcases,"Bush Birmingham Collection Bookcase, Dark Cherry",82517.0,9,0.30,-11788.0,13097.0,10478.0
1,1761,CA-2015-130785,2015-09-05,2015-09-09,Standard Class,AG-10900,Arthur Gainer,Consumer,United States,San Diego,...,FUR-BO-10000330,Furniture,Bookcases,"Sauder Camden County Barrister Bookcase, Plank...",41133.0,4,0.15,-483.0,12098.0,10404.0
2,1595,CA-2015-118423,2015-03-24,2015-03-27,First Class,DP-13390,Dennis Pardue,Home Office,United States,Peoria,...,FUR-BO-10000362,Furniture,Bookcases,Sauder Inglewood Library Bookcases,35905.0,3,0.30,-3590.0,17098.0,13165.0
3,1933,CA-2017-161200,2017-08-06,2017-08-10,Second Class,SV-20365,Seth Vernon,Consumer,United States,Lafayette,...,FUR-BO-10000468,Furniture,Bookcases,O'Sullivan 2-Shelf Heavy-Duty Bookcases,14574.0,3,0.00,2331.0,4858.0,4081.0
4,4089,US-2014-156559,2014-08-19,2014-08-26,Standard Class,LH-16900,Lena Hernandez,Consumer,United States,Jonesboro,...,FUR-BO-10000711,Furniture,Bookcases,"Hon Metal Bookcases, Gray",63882.0,9,0.00,17248.0,7098.0,5182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,3244,CA-2017-113355,2017-12-01,2017-12-05,Standard Class,SJ-20215,Sarah Jordon,Consumer,United States,Grand Prairie,...,TEC-PH-10004912,Technology,Phones,Cisco SPA112 2 Port Phone Adapter,21980.0,5,0.20,2472.0,5495.0,3901.0
1890,1659,CA-2017-161809,2017-01-20,2017-01-26,Standard Class,TH-21100,Thea Hendricks,Consumer,United States,Los Angeles,...,TEC-PH-10004922,Technology,Phones,RCA Visys Integrated PBX 8-Line Router,16077.0,3,0.20,1004.0,6698.0,5024.0
1891,1902,CA-2016-151141,2016-08-20,2016-08-23,First Class,DW-13480,Dianna Wilson,Home Office,United States,Detroit,...,TEC-PH-10004924,Technology,Phones,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",1478.0,2,0.00,399.0,739.0,539.0
1892,1860,CA-2015-121608,2015-09-03,2015-09-05,First Class,JB-15400,Jennifer Braxton,Corporate,United States,Bristol,...,TEC-PH-10004959,Technology,Phones,Classic Ivory Antique Telephone ZL1810,10049.0,1,0.00,2512.0,10049.0,7537.0


In [43]:
store_product_df['ProductID'].str[-4:].max()

'5031'

## Order_Details

In [44]:
order_details_df=store_copied[["OrderID","ProductID","Discount","Quantity"]]

In [45]:
order_details_f=order_details_df.drop_duplicates(subset=["OrderID", "ProductID"], ignore_index=True)

In [46]:
#We need to make a new column to use as a primary key for Order_details
order_details_f['OrderDetailsID'] = range(1, len(order_details_f) + 1)

C:\Users\godwi\AppData\Local\Temp\ipykernel_21808\1664859533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_details_f['OrderDetailsID'] = range(1, len(order_details_f) + 1)


In [47]:
order_details_f['OrderDetailsID'] = "OD" + order_details_f['OrderDetailsID'].astype(str).str.zfill(6)

C:\Users\godwi\AppData\Local\Temp\ipykernel_21808\675396662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_details_f['OrderDetailsID'] = "OD" + order_details_f['OrderDetailsID'].astype(str).str.zfill(6)


In [48]:
order_details_f[["OrderDetailsID","OrderID","ProductID","Discount","Quantity"]]

,OrderDetailsID,OrderID,ProductID,Discount,Quantity
0,OD000001,CA-2016-152156,FUR-BO-10001798,0.00,2
1,OD000002,CA-2016-152156,FUR-CH-10000454,0.00,3
2,OD000003,CA-2016-138688,OFF-LA-10000240,0.00,2
3,OD000004,US-2015-108966,FUR-TA-10000577,0.45,5
4,OD000005,US-2015-108966,OFF-ST-10000760,0.20,2
...,...,...,...,...,...
9981,OD009982,CA-2014-110422,FUR-FU-10001889,0.20,3
9982,OD009983,CA-2017-121258,FUR-FU-10000747,0.00,2
9983,OD009984,CA-2017-121258,TEC-PH-10003645,0.20,2
9984,OD009985,CA-2017-121258,OFF-PA-10004041,0.00,4


# Export to CSV

In [49]:
### customers

import pandas as pd

# Assuming customers_df is your DataFrame
customers = customer_f[["CustomerID","CustomerName","Segment"]]

# Define the file path for the CSV export
csv_file_path = "customers.csv"

# Export the DataFrame to a CSV file
customers.to_csv(csv_file_path, index=False)

print("customers exported to CSV successfully.")


customers exported to CSV successfully.


In [50]:
### address

import pandas as pd

# Assuming customers_df is your DataFrame
address = address_f[["AddressID","CustomerID","Country","Region","State","City","PostalCode"]]

# Define the file path for the CSV export
csv_file_path = "address.csv"

# Export the DataFrame to a CSV file
address.to_csv(csv_file_path, index=False)

print("customers exported to CSV successfully.")


customers exported to CSV successfully.


In [51]:
## orders

import pandas as pd

# Assuming orders_df is your DataFrame
orders = store_order_no_duplicates[["OrderID","OrderDate","ShipDate","ShipMode","AddressID","CustomerID"]]

# Define the file path for the CSV export
csv_file_path = "orders.csv"

# Export the DataFrame to a CSV file
orders.to_csv(csv_file_path, index=False)

print("orders exported to CSV successfully.")


orders exported to CSV successfully.


In [52]:

## Products



products = store_product_df[["ProductID","ProductName", "Category","SubCategory","ProductPrice","SupplierPrice"]]

# Define the file path for the CSV export
csv_file_path = "products.csv"

# Export the DataFrame to a CSV file
products.to_csv(csv_file_path, index=False)

print("products exported to CSV successfully.")


products exported to CSV successfully.


In [53]:
products

,ProductID,ProductName,Category,SubCategory,ProductPrice,SupplierPrice
0,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry",Furniture,Bookcases,13097.0,10478.0
1,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases,12098.0,10404.0
2,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases,17098.0,13165.0
3,FUR-BO-10000468,O'Sullivan 2-Shelf Heavy-Duty Bookcases,Furniture,Bookcases,4858.0,4081.0
4,FUR-BO-10000711,"Hon Metal Bookcases, Gray",Furniture,Bookcases,7098.0,5182.0
...,...,...,...,...,...,...
1889,TEC-PH-10004912,Cisco SPA112 2 Port Phone Adapter,Technology,Phones,5495.0,3901.0
1890,TEC-PH-10004922,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,6698.0,5024.0
1891,TEC-PH-10004924,"SKILCRAFT Telephone Shoulder Rest, 2"" x 6.5"" x...",Technology,Phones,739.0,539.0
1892,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones,10049.0,7537.0


In [54]:
## Order_details



order_details = order_details_f[["OrderDetailsID","OrderID","ProductID","Discount","Quantity"]]

# Define the file path for the CSV export
csv_file_path = "order_details.csv"

# Export the DataFrame to a CSV file
order_details.to_csv(csv_file_path, index=False)

print("order_details exported to CSV successfully.")


order_details exported to CSV successfully.


# Creation of DataBase

In [55]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')
cursor = conn.cursor()

# Create customers table with unique CustomerID
cursor.execute('''CREATE TABLE customers (
                    CustomerID VARCHAR PRIMARY KEY,
                    CustomerName VARCHAR,
                    Segment TEXT,
                    UNIQUE(CustomerID)  -- Ensure CustomerID is unique
                )''')

# Create address table with foreign key reference to customers and unique AddressID
cursor.execute('''CREATE TABLE address (
                    AddressID VARCHAR PRIMARY KEY,
                    CustomerID VARCHAR,
                    Country VARCHAR,
                    Region VARCHAR,
                    State VARCHAR,
                    City VARCHAR,
                    PostalCode INTEGER,
                    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
                    UNIQUE(AddressID)  -- Ensure AddressID is unique
                )''')

# Create products table with unique ProductID
cursor.execute('''CREATE TABLE products (
                    ProductID VARCHAR PRIMARY KEY,
                    ProductName VARCHAR,
                    Category TEXT,
                    SubCategory TEXT,
                    ProductPrice REAL,                 
                    SupplierPrice REAL,
                    UNIQUE(ProductID)  -- Ensure ProductID is unique
                )''')

# Create orders table with primary key, foreign key reference to customers, and unique OrderID
cursor.execute('''CREATE TABLE orders (
                    OrderID VARCHAR PRIMARY KEY,
                    OrderDate DATE,
                    ShipDate DATE,
                    ShipMode TEXT,
                    AddressID VARCHAR,
                    CustomerID VARCHAR,                   
                    FOREIGN KEY (AddressID) REFERENCES addresss(AddressID),
                    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID)
                    UNIQUE(OrderID)  -- Ensure OrderID is unique
                )''')



	
# Create order_details table with primary key, foreign key reference to customers, orders and products
cursor.execute('''CREATE TABLE order_details (
                    OrderDetailsID VARCHAR PRIMARY KEY, 
                    OrderID VARCHAR, 
                    ProductID VARCHAR,
                    Discount REAL,
                    Quantity INTEGER,
                    FOREIGN KEY (OrderID) REFERENCES orders(OrderID),
                    FOREIGN KEY (ProductID) REFERENCES products(ProductID)
                )''')

# Commit changes and close connection
conn.commit()
conn.close()


# Populating of DataBase with DataFrame

In [56]:
##customer

import sqlite3
customers=customer_f[["CustomerID","CustomerName","Segment"]] 
# Connect to the SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')

# Write DataFrames to SQLite database
customers.to_sql('customers', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()




In [57]:
## address

import sqlite3
# Connect to SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')


# Extract specific columns into a new DataFrame
address=address_f[["AddressID","CustomerID","Country","Region","State","City","PostalCode"]]# Write the DataFrame to the SQLite database
address.to_sql('address', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()

In [58]:
## Orders
orders= store_order_no_duplicates[["OrderID","OrderDate","ShipDate","ShipMode","AddressID","CustomerID"]]
# Connect to the SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')

# Write DataFrames to SQLite database
orders.to_sql('orders', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()




In [59]:
## Products

# Connect to SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')

# Extract specific columns into a new DataFrame
products=store_product_df[["ProductID","ProductName", "Category","SubCategory","ProductPrice","SupplierPrice"]]# Write the DataFrame to the SQLite database
products.to_sql('products', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()


In [60]:

##Order_details

# Connect to SQLite database
conn = sqlite3.connect('VS_GROUP_DataBase.db')


# Extract specific columns into a new DataFrame
order_details=order_details_f[["OrderDetailsID","OrderID","ProductID","Discount","Quantity"]]
order_details.to_sql('order_details', conn, if_exists='append', index=False)

# Commit changes and close connection
conn.commit()
conn.close()


  ## SOME Queries

In [61]:
conn = sqlite3.connect('VS_GROUP_DataBase.db')

In [62]:
cursor = conn.cursor()

In [63]:
# Perform a query

cursor.execute("SELECT * FROM Customers WHERE Segment = 'Corporate'")
rows = cursor.fetchall()

In [64]:
# Convert the retrieved data into a Dataframe
Corporate_customers = pd.DataFrame([row[0:2] for row in rows], columns= ['CustomerID', 'CustomerName'])

In [65]:
Corporate_customers

,CustomerID,CustomerName
0,DV-13045,Darrin Van Huff
1,KB-16585,Ken Black
2,GH-14485,Gene Hale
3,LC-16930,Linda Cazamias
4,RA-19885,Ruben Ausman
...,...,...
231,TC-21145,Theresa Coyne
232,CM-12715,Craig Molinari
233,FW-14395,Fred Wasserman
234,HE-14800,Harold Engle


In [66]:
# Close the connection
conn.close()

In [67]:
conn = sqlite3.connect('VS_GROUP_DataBase.db')
cursor = conn.cursor()
cursor.execute("SELECT Category, SUM((ProductPrice * (1 - Discount)) * Quantity) AS TotalSales FROM products P JOIN order_details OD ON P.ProductID = OD.ProductID GROUP BY Category ORDER BY TotalSales DESC LIMIT 1")
rows = cursor.fetchall()

In [69]:
# Convert the retrieved data into a Dataframe
High_Category = pd.DataFrame([row[0:2] for row in rows], columns= ['Category','TotalSales'])

In [70]:
High_Category

,Category,TotalSales
0,Technology,81439500.8
